In [1]:
import copy
import time

from lerobot.common.robot_devices.motors.dynamixel import DynamixelMotorsBus, TorqueMode


%load_ext autoreload
%autoreload 2

In [2]:
leader_port = "/dev/tty.usbmodem58760436131"
follower_port = "/dev/tty.usbmodem58760433231"

leader_arm = DynamixelMotorsBus(
    port=leader_port,
    motors={
        # name: (index, model)
        "shoulder_pan": (1, "xl330-m077"),
        "shoulder_lift": (2, "xl330-m077"),
        "elbow_flex": (3, "xl330-m077"),
        "wrist_flex": (4, "xl330-m077"),
        "wrist_roll": (5, "xl330-m077"),
        "gripper": (6, "xl330-m077"),

    },
)

follower_arm = DynamixelMotorsBus(
    port=follower_port,
    motors={
        # name: (index, model)
        "shoulder_pan": (1, "xl430-w250"),
        "shoulder_lift": (2, "xl430-w250"),
        "elbow_flex": (3, "xl330-m288"),
        "wrist_flex": (4, "xl330-m288"),
        "wrist_roll": (5, "xl330-m288"),
        "gripper": (6, "xl330-m288"),
    },
)


In [4]:
from lerobot.common.robot_devices.robots.manipulator import ManipulatorRobot

robot = ManipulatorRobot(
    robot_type="koch",
    leader_arms={"main": leader_arm},
    follower_arms={"main": follower_arm},
    calibration_dir="../.cache/calibration/koch",
)
robot.connect()


Connecting main follower arm.
Connecting main leader arm.
Activating torque on main follower arm.
[  1.40625   135.8789    171.73828    22.41211    -6.2402344  40.253906 ]
[  0.        128.67188   175.69336    -2.109375   -1.3183594  28.03711  ]


In [15]:
leader_pos = robot.leader_arms["main"].read("Present_Position")
follower_pos = robot.follower_arms["main"].read("Present_Position")


print([f"{x:.2f}" for x in leader_pos])
print([f"{x:.2f}" for x in follower_pos])

['-1.05', '126.39', '175.69', '-2.81', '-1.23', '29.27']
['0.00', '128.67', '175.78', '-2.11', '-1.32', '28.04']


In [10]:
leader_pos

array([ 8.7890625e-02,  1.3587891e+02,  1.7165039e+02,  2.2412109e+01,
       -5.9765625e+00,  4.0253906e+01], dtype=float32)

In [ ]:
array([-0.43945312, 133.94531, 179.82422, -18.984375, -1.9335938, 34.541016], dtype=float32)
array([-0.58723712, 131.72314, 174.98743, -16.872612, 0.786213, 35.271973], dtype=float32)

## Tele-op

In [3]:
from lerobot.common.robot_devices.robots.manipulator import ManipulatorRobot

robot = ManipulatorRobot(
    robot_type="koch",
    leader_arms={"main": leader_arm},
    follower_arms={"main": follower_arm},
    calibration_dir="../.cache/calibration/koch",
)
robot.connect()


Connecting main follower arm.
Connecting main leader arm.
Activating torque on main follower arm.


In [4]:
leader_pos = robot.leader_arms["main"].read("Present_Position")
follower_pos = robot.follower_arms["main"].read("Present_Position")
print(leader_pos)
print(follower_pos)


[  0.       135.43945  177.71484    6.591797  -5.888672  18.632812]
[ -0.87890625 172.35352    174.99023     -1.0546875   -3.7792969
  24.169922  ]


In [5]:
robot.disconnect()

## Read and write with DynamixelMotorsBus

In [5]:
leader_arm.connect()

In [6]:
leader_arm.write("Torque_Enable", TorqueMode.ENABLED.value)

In [6]:
leader_arm.write("Torque_Enable", TorqueMode.DISABLED.value)

In [12]:
leader_pos = leader_arm.read("Present_Position")
print(f"Curr pos: {leader_pos}")

leader_pos_new = copy.copy(leader_pos)
leader_pos_new[1] += 10

print(f"Next pos: {leader_pos_new}")
leader_arm.write("Goal_Position", leader_pos_new)
time.sleep(1)

leader_pos_read = leader_arm.read("Present_Position")
delta_pos = leader_pos_read - leader_pos
print(f"delta pos: {delta_pos}")

Curr pos: [2062 1274 2400 1971 2171 1914]
Next pos: [2062 1284 2400 1971 2171 1914]
delta pos: [ 0  4 -1  0  0  0]


In [3]:
follower_arm.connect()

In [18]:
follower_arm.write("Torque_Enable", TorqueMode.DISABLED.value)

In [19]:
follower_arm.write("Torque_Enable", TorqueMode.ENABLED.value)


In [26]:
follower_pos = follower_arm.read("Present_Position")
print(f"Curr pos: {follower_pos}")

follower_pos_new = copy.copy(follower_pos)
follower_pos_new[3] += 10

print(f"Next pos: {follower_pos_new}")
follower_arm.write("Goal_Position", follower_pos_new)
time.sleep(1)

follower_pos_read = follower_arm.read("Present_Position")
delta_pos = follower_pos_read - follower_pos
print(f"delta pos: {delta_pos}")

Curr pos: [2195 1305 1531 1046  655 2190]
Next pos: [2195 1305 1531 1056  655 2190]
delta pos: [  0   0 -46   0   0   0]


In [28]:
# For some reason, the 3rd motor is always moving even though we only set the first motor to move
follower_pos = follower_arm.read("Present_Position")
print(f"Curr pos: {follower_pos}")


Curr pos: [2195 1305 1485 1046  655 2190]


In [14]:
follower_arm.write("Torque_Enable", TorqueMode.ENABLED.value)

In [40]:
# Get the current position
position = follower_arm.read("Present_Position")
print(position)
# Update first motor (shoulder_pan) position by +10 steps
position[0] -= 10
follower_arm.write("Goal_Position", position)


[1990 1111 1190  709 2108 1989]


In [42]:
follower_arm.write("Torque_Enable", TorqueMode.DISABLED.value)
leader_arm.disconnect()
follower_arm.disconnect()